In [ ]:
pip install imbalanced-learn

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import time
import warnings
import numpy as np
from nltk.corpus import stopwords
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
#from imblearn.over_sampling import SMOTE
from collections import Counter
from scipy.sparse import hstack
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from collections import Counter, defaultdict
from sklearn.calibration import CalibratedClassifierCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import math
from sklearn.metrics import normalized_mutual_info_score
from sklearn.ensemble import RandomForestClassifier
warnings.filterwarnings("ignore")

from mlxtend.classifier import StackingClassifier
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression

In [ ]:
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2024-03-19 11:35:06--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.1’

book-crossings.zip. 100%[===================>]  24.88M   144MB/s    in 0.2s    

2024-03-19 11:35:06 (144 MB/s) - ‘book-crossings.zip.1’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
df_books.head(10)

In [ ]:
df_books.tail(10)

In [ ]:
df_books.shape

In [ ]:
df_books.info()

In [ ]:
df_ratings.head(10)

In [ ]:
df_ratings.tail(10)

In [ ]:
df_ratings.info()

In [ ]:
 # book_ratings = df_ratings['user'].value_counts()
  #book_ratings.sort_values(ascending = False).head()
  #len(book_ratings[book_ratings<200])
  #df_ratings['user'].isin(book_ratings[book_ratings<200].index).sum()

  # Count the number of ratings per user
book_ratings = df_ratings['user'].value_counts()
print("User ratings counts:\n", book_ratings)

# Get the top users with the highest number of ratings
top_users = book_ratings.sort_values(ascending=False).head()
print("Top users with highest ratings:\n", top_users)

# Count the number of users with fewer than 200 ratings
users_below_threshold = df_ratings['user'].isin(book_ratings[book_ratings < 200].index).sum()
print("Number of users with fewer than 200 ratings:", users_below_threshold)

In [ ]:
df_ratings_books = df_ratings[
    ~df_ratings['user'].isin(book_ratings[book_ratings<200].index)
]
print("Shape of filtered DataFrame:",df_ratings_books.shape)

In [ ]:
df_ratings_books.info() #so,here we can see that the entries reduced to 527556 which shows number of users with less than 200 ratings.

Remove books with less than 100 ratings.

In [ ]:
book_ratings_isbn = df_ratings['isbn'].value_counts()
book_ratings_isbn.sort_values(ascending=False).head()
print("Top ISBNs with highest ratings:\n", book_ratings_isbn)

In [ ]:
len(book_ratings_isbn[book_ratings_isbn<100])

In [ ]:
df_books['isbn'].isin(book_ratings_isbn[book_ratings_isbn<100].index).sum()

In [ ]:
df_ratings_books = df_ratings_books[
    ~df_ratings_books['isbn'].isin(book_ratings_isbn[book_ratings_isbn<100].index)
]
print("Shape of filtered DataFrame:",df_ratings_books.shape)

In [ ]:
df_ratings_books.head(10)

KNN

In [ ]:
df = df_ratings_books.pivot_table(index=['user'],columns=['isbn'],values='rating').fillna(0)
df = df.T
df.head()

In [ ]:
df.index = df.join(df_books.set_index('isbn'))['title']
df.index.name = 'title'

In [ ]:
df  = df.sort_index()
df.head()

In [ ]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(metric ='cosine') #cosine distance metric will be used to measure the similarity between the points.
model.fit(df.values)

In [ ]:
title = 'The Queen of the Damned (Vampire Chronicles (Paperback))'
df.loc[title].shape

In [ ]:
distance, indice = model.kneighbors([df.loc[title].values],n_neighbors=6)

print(distance)
print(indice)

In [ ]:
df.iloc[indice[0]].index.values

In [ ]:
pd.DataFrame({
    'title'   : df.iloc[indice[0]].index.values,
    'distance': distance[0]
}) \
.sort_values(by='distance', ascending=False)

In [ ]:
 #function to return recommended books - this will be tested
def get_recommends(title = ""):
  try:
    book = df.loc[title]
  except KeyError as e:
    print('The given book', e, 'does not exist')
    return

  distance, indice = model.kneighbors([book.values], n_neighbors=6)

  recommended_books = pd.DataFrame({
      'title'   : df.iloc[indice[0]].index.values,
      'distance': distance[0]
    }) \
    .sort_values(by='distance', ascending=False) \
    .head(5).values

  return [title, recommended_books]

In [ ]:
books = get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()